This notebook can be run on MP data only, as they already contain the formation energeis per atom. We did not calculate these for the experimental 3DCD data set. 

In [ ]:
from data import DATA_3DCD, DATA_MP

In [ ]:
%run ./modules.ipynb
sns.set(style="white", palette="muted", color_codes=True)
my_c = colors.ListedColormap(["mediumblue", "red"])
c = ["mediumblue", "red"]
sns.set_palette(sns.color_palette(c))

LOADING FORMATION ENERGIES

In [ ]:
from pathlib import Path

df = pd.read_csv(DATA_MP.data)

In [ ]:
npzfile = np.load(DATA_MP.soap, allow_pickle=True)

# TODO: FIX THE INDEX MAPPING!!!
idx = npzfile["idx"]
# idx_ = idx[idx < len(idx) - 1]
df = df.loc[idx]
# df = df[:-1]
######


df["magic"] = npzfile["magic"]

In [ ]:
f, ax = plt.subplots(figsize=(9, 8))
sns.histplot(
    data=df,
    x="e_form",
    hue="magic",
    multiple="dodge",
    legend=False,
    common_norm=False,
    stat="probability",
    kde=True,
)
sns.despine(left=True, bottom=True)
plt.xlabel("Formation Energy per atom (eV)")

In [ ]:
X_raw = npzfile["X_raw"]
y_raw = y_raw = np.array(df["e_form"]).reshape(-1, 1)
y_atom = npzfile["magic"]
my_frames = npzfile["my_frames"]

PRINCIPAL COVARIATES REGRESSION (PCovR) REPRESENTATION

In [ ]:
i_train, i_test, X_train, X_test, y_train, y_test = train_test_split(
    np.arange(X_raw.shape[0]), X_raw, y_raw, train_size=0.8
)
print(f"Shape of testing data is: {X_train.shape}, ||X|| = {np.linalg.norm(X_train)}.")
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

print(
    "Norm of the features: %0.2f \nNorm of the targets: %0.2f"
    % (np.linalg.norm(X_raw), np.linalg.norm(y_raw))
)
x_scaler = StandardFlexibleScaler(column_wise=False).fit(X_raw)
y_scaler = StandardFlexibleScaler(column_wise=True).fit(y_raw)

# Center total dataset
X = x_scaler.transform(X_raw)
Y = y_scaler.transform(y_raw)

# Center training data
X_train = x_scaler.transform(X_train)
y_train = y_scaler.transform(y_train)

# Center training data
X_test = x_scaler.transform(X_test)
y_test = y_scaler.transform(y_test)

print("After normalization: ")
print(f"Shape of testing data is: {X_train.shape}, ||X|| = {np.linalg.norm(X_train)}.")
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

print(
    "Norm of the features: %0.2f \nNorm of the targets: %0.2f"
    % (np.linalg.norm(X_raw), np.linalg.norm(y_raw))
)

GRID CV REGRESSION

In [ ]:
alphas = np.logspace(-0, -6, 30)
ridgecv = RidgeCV(cv=4, fit_intercept=False, alphas=alphas)
ridgecv.fit(X_train, y_train)
yp_test = ridgecv.predict(X_test)
yp_train = ridgecv.predict(X_train)
print("R^2 of training is ", ridgecv.score(X_train, y_train))
print("R^2 of testing is ", ridgecv.score(X_test, y_test))
print("Best regularization is ", ridgecv.alpha_)

fig, (ax_train, ax_test) = plt.subplots(1, 2, figsize=(8, 4), sharex=True, sharey=True)
ax_train.scatter(y_train, yp_train, c=np.abs(y_train - yp_train), cmap="bone_r")
ax_test.scatter(y_test, yp_test, c=np.abs(y_test - yp_test), cmap="bone_r")

bounds = (
    np.min((Y.min(), yp_train.min(), yp_test.min())),
    np.max((Y.max(), yp_train.max(), yp_test.max())),
)

ax_train.plot(bounds, bounds, "r--")
ax_train.set_title("Training set")
ax_test.set_title("Testing set")
ax_test.plot(bounds, bounds, "r--")
sns.despine(left=True, bottom=True)

print("RMSE on predicted quantity = ", mean_squared_error(y_test, yp_test))

HOW MIXING PARAMETER BETA (BETWEEN PCA AND REGRESSION) AFFECTS PCOVR

In [ ]:
betas = [0, 0.25, 0.5, 0.75, 1]

fig, axes = plt.subplots(2, 5, figsize=(17, 10))
sns.despine(left=True, bottom=True)

for i in range(len(betas)):

    pcovr = PCovR(mixing=betas[i], n_components=2)
    pcovr.fit(X, Y)
    T = pcovr.transform(X)
    yp = pcovr.predict(X)

    axes[0, i].scatter(
        T[:, 0],
        T[:, 1],
        s=50,
        alpha=0.3,
        c=y_atom,
        cmap=colors.ListedColormap(["mediumblue", "red"]),
    )
    axes[0, i].set_xlabel(r"$PC_1$")
    axes[0, i].set_ylabel(r"$PC_2$")

    axes[1, i].scatter(Y, yp, s=50, alpha=0.7, c=np.abs(Y - yp), cmap="bone_r")
    axes[1, i].plot(axes[1, i].get_xlim(), axes[1, i].get_xlim(), "r--")
    axes[1, i].set_xlabel(r"y")
    axes[1, i].set_ylabel(r"predicted y")

    axes[0, i].set_title(r"$\beta=$" + str(betas[i]))
    fig.tight_layout()

ENERGY DESCRIPTORS 

In [ ]:
df.head()
e_form = df["e_form"]
e_hull = df["e_hull"]
e_gap = df["gap pbe"]

In [ ]:
n_PC = 5  #  number of principal covariates
titles = [
    "Formation energy per atom (eV)",
    "Energy above convex hull (eV)",
    "Band gab energy (eV)",
]
quantity = [e_form, e_hull, e_gap]
for i in range(len(titles)):
    X = np.copy(X_raw)
    y_raw = np.array(quantity[i]).reshape(-1, 1)
    i_train, i_test, X_train, X_test, y_train, y_test = train_test_split(
        np.arange(X.shape[0]), X, y_raw, train_size=0.8
    )

    x_scaler = StandardFlexibleScaler(column_wise=False).fit(X)
    y_scaler = StandardFlexibleScaler(column_wise=True).fit(y_raw)

    # Center total dataset
    X = x_scaler.transform(X)
    Y = y_scaler.transform(y_raw)

    # Center training data
    X_train = x_scaler.transform(X_train)
    y_train = y_scaler.transform(y_train)

    # Center training data
    X_test = x_scaler.transform(X_test)
    y_test = y_scaler.transform(y_test)
    pcovr = PCovR(mixing=0.5, n_components=n_PC)
    pcovr.fit(X, Y)

    T = pcovr.transform(X)
    yp = pcovr.predict(X)

    ridgecv.fit(X_train, y_train)
    yp_test = ridgecv.predict(X_test)

    # SCATTER PLOTS
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 4), sharex=True, sharey=True)
    ax1.scatter(T[:, 0], T[:, 1], alpha=0.4, c=y_atom, cmap=my_c)
    g = ax2.scatter(T[:, 0], T[:, 1], alpha=0.4, c=Y, cmap="plasma")
    plt.colorbar(g, orientation="vertical", label=titles[i])
    ax1.set_xlabel(r"$PCov_1$")
    ax2.set_xlabel(r"$PCov_1$")
    ax1.set_ylabel(r"$PCov_2$")
    fig.tight_layout()
    sns.despine(left=True, bottom=True)

    # LINE PLOT
    df = pd.DataFrame(T)
    g = sns.jointplot(
        data=df, x=df[0], y=df[1], hue=y_atom.ravel(), kind="kde", legend=False
    )
    g.set_axis_labels(r"$PCov_1$", r"$PCov_2$")
    fig.tight_layout()
    sns.despine(left=True, bottom=True)

    print("R^2 with PCovR= ", pcovr.score(X, Y))
    print("R^2 of testing is with RidgeCV", ridgecv.score(X_test, y_test))
    print("RMSE on predicted quantity = ", mean_squared_error(y_test, yp_test))
    print(
        "error in units (eV) of predicted quantity before scaling = ",
        (max(y_raw) - min(y_raw)) * (1 - ridgecv.score(X_test, y_test)),
    )
    print(
        "error in units (eV) of predicted quantity after scaling= ",
        (max(Y) - min(Y)) * (1 - ridgecv.score(X_test, y_test)),
    )

    plt.show()
    plt.close()